In [1]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings("ignore")

# Custom modules and functions
import capstone.portfolio.optimize as opt
from capstone.portfolio.prune import prune_recommended_portfolios
from capstone.model_selection import overunder_error
from capstone.utils import read_file, get_sectors

# Machine learning and modeling tools
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor

# Progress bar for loops
from tqdm.auto import tqdm

# Set visualization style and adjust plot settings
sns.set_style("whitegrid")
plt.rcParams["lines.linewidth"] = 1
plt.rcParams["axes.edgecolor"] = "k"

In [2]:
# Load files
df = read_file("master_df", index_col="Date")
snp_log_returns = read_file("snp_log_returns", index_col="Date")
stocks_by_sector = read_file("stocks_by_sector", index_col=0)
sectors = get_sectors()

In [3]:
# Separate the combined dataframe into targets (sector average returns) and features
y_all = df[sectors]
X_all = df[df.columns[~df.columns.isin(y_all.columns)]]

X_all.shape, y_all.shape

((4230, 109), (4230, 11))

In [4]:
# Construct a pipeline for PCA with standard scaling
pca_pipe = make_pipeline(StandardScaler(), PCA(n_components=.8, random_state=42))

# Store pca-transformed features in a dataframe
X_pca = pd.DataFrame(
    pca_pipe.fit_transform(X_all), 
    index=X_all.index
)

# Rename columns
X_pca.columns = [f"PC{i+1}" for i in X_pca.columns]

X_pca.tail()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26
Date,,,,,,,,,,,,,,,,,,,,,
2023-09-05,1.572267,2.251625,-0.166538,0.577534,1.593571,0.727317,-1.045755,1.717850,-1.269176,0.550307,...,-0.728028,0.924053,-0.644267,1.030472,-0.345593,-0.441153,1.619234,0.391941,-0.633839,-0.080373
2023-09-06,0.149746,4.029874,-2.027692,-0.273766,2.112711,-1.341697,-1.386221,1.648393,-1.603688,0.135565,...,-0.506765,-0.260769,-0.553688,0.134524,-0.372484,-0.691424,1.085630,1.428667,0.183519,-0.880374
2023-09-07,-4.043578,-0.026559,1.689399,-1.105289,2.137391,1.417627,-1.899516,1.022726,-0.914299,0.664205,...,1.818843,-0.777557,0.211620,-1.885068,-0.761534,0.386206,0.876800,0.946822,0.448153,-0.360191
2023-09-08,-3.933137,6.109104,0.671842,0.143338,-0.723764,0.634047,-2.434709,-0.209488,-1.234175,-0.387969,...,-1.609450,-0.543206,0.077936,0.335162,-1.065407,0.129781,0.520862,-0.116500,0.964564,0.432200
2023-09-11,-2.804780,6.085291,-0.323564,0.342325,-2.640603,1.833748,-0.759626,0.070136,-2.033351,-0.796932,...,-2.936066,2.195284,-0.697539,-0.705440,0.354325,-1.076233,-0.243180,-0.156890,1.308446,-0.409407


In [5]:
# Define models
models = {
    # ElasticNet combines L1 and L2 regularization, suitable for feature selection and dealing with multicollinearity.
    'ElasticNet': make_pipeline(StandardScaler(), ElasticNet(alpha=1, l1_ratio=0.5, random_state=42)),
    
    # Support Vector Regressor (SVR) with an RBF kernel can capture non-linear relationships in stock returns.
    'SVR': make_pipeline(StandardScaler(), SVR(kernel='rbf', C=1, gamma='auto')),
    
    # RandomForestRegressor is an ensemble method that can capture complex relationships and feature importance.
    'RandomForest': RandomForestRegressor(n_estimators=100, max_depth=3, random_state=42),
    
    # GradientBoostingRegressor is another ensemble method suitable for capturing non-linear relationships and trends.
    'GradientBoost': GradientBoostingRegressor(n_estimators=100, random_state=42),
    
    # XGBoostRegressor is an optimized gradient boosting algorithm known for its speed and performance.
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
}

In [6]:
# Set the forecast horizon to 126 days (approx. 1/2 trading year)
forecast = 126

# Initialize TimeSeriesSplit object for cross-validation with 2 splits
cv = 2
tscv = TimeSeriesSplit(cv)

# Shift the PCA-transformed features to align with the forecast horizon and remove NA rows
X_pca_shifted = X_pca.shift(forecast).dropna()

# Select the most recent subset of data, doubled to the size of the forecast window
X_pca_recent = X = X_pca_shifted.iloc[-forecast*2:]
y_recent = y_all.iloc[-forecast*2:]

# Initialize dictionary to store the Mean Over-Under Loss (MEAN_OUL) for each model and sector
ouls = {model: pd.DataFrame(index=sectors, columns=["MEAN_OUL"]) for model in models.keys()}

# Loop through each sector to train and validate models
for sector in sectors:
    y = y_recent[sector]

    # Loop through each machine learning model
    for name, model in models.items():
        cv_oul = []  # List to store cross-validation Over-Under Loss for current model

        # Time-series cross-validation
        for train_idx, test_idx in tscv.split(X_pca_recent):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            # Fit the model on the training set and predict on the test set
            model.fit(X_train, y_train)
            y_hat_val = model.predict(X_test)

            # Calculate Over-Under Loss and append to cv_oul list
            cv_oul.append(overunder_error(y_test, y_hat_val))

        # Calculate mean Over-Under Loss across all cross-validation runs
        oul_mean = np.mean(cv_oul)
        ouls[name].loc[sector, "MEAN_OUL"] = oul_mean

# Compute the mean Over-Under Loss for each model across all sectors
mean_ouls = pd.DataFrame({model: oul.mean() for model, oul in ouls.items()})

# Identify the best model based on the lowest mean Over-Under Loss
best_model = mean_ouls.idxmin(axis=1)[0]

print(best_model)

ElasticNet


In [12]:
# Calculate the date immediately following the last date in the existing target data
future_start = y.index.max() + pd.DateOffset(1)

# Calculate the final date for the forecast period
future_end = future_start + pd.DateOffset(forecast - 1)

# Generate a date range covering the entire forecast period
future_dates = pd.date_range(future_start, future_end)

# Use the last 2*forecast rows of the PCA-transformed features as training data
X_train = X_pca[-forecast*2:-forecast]

# Use the last forecast rows of the recent target data as training data
y_train = y_recent[-forecast:]

# Use the last forecast rows of the PCA-transformed features as test data
X_test = X_pca[-forecast:]

# Fit the best-performing model to the training data
models[best_model].fit(X_train, y_train)

# Use the trained model to predict future returns
predicted_returns = pd.DataFrame(models[best_model].predict(X_test), columns=sectors, index=future_dates)

# Inspect the cumulative log returns of each sector by the end of the forecast period
predicted_returns.cumsum().iloc[-1].sort_values(ascending=False)[:5]

INFORMATION_TECHNOLOGY    0.160892
ENERGY                    0.146670
CONSUMER_DISCRETIONARY    0.089027
COMMUNICATION_SERVICES    0.088246
INDUSTRIALS               0.064115
Name: 2024-01-15 00:00:00, dtype: float64

In [8]:
# Compute mean predicted returns of each sector
mean_predicted_returns = predicted_returns.mean()

# Retrieve the sector with the highest predicted returns
recommended_sector = mean_predicted_returns.idxmax()

print(recommended_sector)

INFORMATION_TECHNOLOGY


In [9]:
# Get the stocks belonging to the recommended sector
available_stocks = stocks_by_sector[stocks_by_sector["GICS Sector"] == recommended_sector]["Symbol"].to_list()

# Filter those stocks to only include those present in the S&P 500 log returns data
recommended_stocks = [stock for stock in available_stocks if stock in snp_log_returns.columns]

# Initialize an equal-weight portfolio for the recommended stocks
weights = np.array([1/len(recommended_stocks)] * len(recommended_stocks))

# Get the recent log returns for the recommended stocks
recent_returns = snp_log_returns[-forecast:][recommended_stocks]

# Calculate the optimal weights for a Maximum Sharpe ratio portfolio using recent returns
max_sharpe_weights = opt.max_sharpe_opt(weights, recent_returns)[0]

# Calculate the optimal weights for a Minimum Variance portfolio using recent returns
min_var_weights = opt.min_var_opt(weights, recent_returns)[0]

# Calculate the optimal weights for a Risk Parity portfolio using recent returns
risk_parity_weights = opt.risk_parity_opt(weights, recent_returns)[0]

In [10]:
# Create pandas Series for each portfolio type (Max Sharpe, Min Variance, Risk Parity)
# The Series have stock symbols as the index and respective weights as values.
max_sharpe_portfolio = pd.Series(max_sharpe_weights, index=recommended_stocks)
min_var_portfolio = pd.Series(min_var_weights, index=recommended_stocks)
risk_parity_portfolio = pd.Series(risk_parity_weights, index=recommended_stocks)

# Prune the portfolios using the 'prune_recommended_portfolios' function
# This removes assets with negligible weights and renormalizes the remaining weights.
max_sharpe_portfolio, min_var_portfolio, risk_parity_portfolio = \
    prune_recommended_portfolios(
        max_sharpe_portfolio, min_var_portfolio, risk_parity_portfolio
    )

In [15]:
# Simulate allocations with starting balance
balance = 100_000

max_sharpe_allocations = max_sharpe_portfolio * balance
min_var_allocations = min_var_portfolio * balance
risk_parity_allocations = risk_parity_portfolio * balance

In [19]:
# Inspect allocations
allocations = [max_sharpe_allocations, min_var_allocations, risk_parity_allocations]
portfolios = ["MAX_SHARPE", "MIN_VAR", "RISK_PARITY"]

for name, allocation in zip(portfolios, allocations):
    print(name)
    print(allocation.sort_values(ascending=False))
    print()

MAX_SHARPE
IBM     33241.310395
AKAM    27024.379626
ORCL    19626.585429
NVDA     7406.296679
ADBE     4079.603037
CRM      3167.185277
INTC     2318.406006
INTU     1732.935974
MSFT     1403.297577
dtype: float64

MIN_VAR
IBM     38405.257609
ROP     20519.111493
VRSN    13991.314677
MSI      6528.583711
MSFT     5395.376317
AAPL     4203.036844
CSCO     4048.467530
CRM      3681.360081
AKAM     2172.345140
CTSH     1055.146599
dtype: float64

RISK_PARITY
MSI     3214.176551
IBM     3198.065181
ROP     3172.596041
VRSN    3086.513404
CSCO    2947.525686
AKAM    2928.564076
AAPL    2749.011148
CTSH    2692.799975
IT      2624.096986
PTC     2620.537527
GLW     2602.510294
CRM     2590.512095
TYL     2558.627617
ORCL    2547.879023
TDY     2536.458611
APH     2458.748296
FICO    2429.993574
MSFT    2417.584984
JNPR    2394.490940
NTAP    2322.778446
ACN     2304.759617
FFIV    2270.361779
GEN     2259.109064
HPQ     2249.696350
TRMB    2017.862442
ANSS    1854.984947
INTU    1793.79509